# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=AqT78YATvDHULsOcdSJk5eVSzU6rPPzFFwutB_wQYeo&tc=TDc_oQJ4L70ypJyT272HHc5xrxFI6yvxZKiVIEvIehg&cc=2nhYSrtfPMlSvYo4SIK7xJFLuRJ4qCDTsCo-kxMWbxA

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgttMJzlN836TLKntuZgrLi17aRBPYissYPVema_JQqv643-skfD5E

Successfully saved authorization token.


In [2]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 9.7 MB/s 
     |████████████████████████████████| 6.3 MB 31.0 MB/s 
     |████████████████████████████████| 16.7 MB 275 kB/s 


In [3]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
climate_stats/         irrig_lbrb/   masked/        seb_change.png
diversion_timeseries/  IrrMapper/    ny_change.png  set_change.png
fu_change.png          LBRB_shp/     output_files/  subset_test_shp/
irrigation_companies/  lcmap_files/  POUs/


In [5]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'POUs/POUs_EDIT_051822_Merge.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [6]:
## --------------------------------------- ##
## 2. IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## --------------------------------------- ##

years = np.arange(1987,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [ ]:
## Ask kendra if I should use this or an automated call for zonal stats, either works
## and both csvs need to be editted to an extent

## could define a spatial reduce function

def spatial_reduction(image_collection, shape_file, reducer, scale, file_name, format, output_folder, name):

  '''
  This function is used to produce zonal statistics out of an image collection in GEE.

  Variables:
  image_collection: A time series of ee.ImageCollection
  shapefile: The spatial area to reduce the image to. 
  reducer: The ee.Reducer to calculate. It can be a mixture of reducers or just one.
  scale: Pixel size
  file_name: The output table file name.
  format: The output table format, can include 'CSV', "GeoJSON", "KML", "KMZ", or "SHP", or "TFRecord".
  output_folder: The file path for the output file. 
  name: The climate statistic you are calculating (i.e., 'precip_mean')
  '''
  reduce_step = ee.FeatureCollection(image_collection.map(lambda image: image.reduceRegions(collection=shape_file,
                                                                                  scale=scale,
                                                                                  reducer=reducer)))
  flat_reduction = reduce_step.flatten()
  task = ee.batch.Export.table.toDrive(collection = flat_reduction,
                                     description=file_name,
                                     fileNamePrefix=file_name,
                                     fileFormat=format,
                                     folder=output_folder
  )
  task.start()
  while(True):
    if 'COMPLETED' not in task.status().values():
      print(task.status())
      time.sleep(10)
    else:
      file_path = os.path.join(output_folder+'/'+file_name+'.csv')
      df = pd.read_csv(file_path)
      df = df.drop(columns=['.geo', 'Shape_Area', 'Shape_Leng'], axis=1)
      df['Year'] = df['system:index'].str.slice(start=0, stop=4)
      df = df.drop('system:index', axis=1).rename(columns={'mean':name})
      break 

  return df

In [ ]:
new_df = spatial_reduction(image_collection=means_max_temp, 
                  shape_file = subset,
                  reducer = ee.Reducer.mean(),
                  scale=1000,
                  file_name = 'function_try',
                  format = 'CSV',
                  output_folder = 'climate_stats',
                  name = 'max_temp_mean')

{'state': 'READY', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077709471, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}
{'state': 'RUNNING', 'description': 'function_try', 'creation_timestamp_ms': 1653077709471, 'update_timestamp_ms': 1653077717786, 'start_timestamp_ms': 1653077717750, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'ICAP2LDAVQ3GJGL3ZZQXFQL6', 'name': 'projects/earthengine-legacy/operations/ICAP2LDAVQ3GJGL3ZZQXFQL6'}


In [7]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1986,2022)
sum_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-10-01'), (str((i+1))+'-9-30')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [8]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1986,2022)
irrig_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-03-15'), (str((i+1))+'-10-15')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':str(i)}) #select the bands to analyze
  irrig_pr.append(prcp) #calculate the mean across all pixels

irrig_precip = ee.ImageCollection(irrig_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [ ]:
new_df = spatial_reduction(image_collection=sum_precip, 
                  shape_file = subset,
                  reducer = ee.Reducer.mean(),
                  scale=1000,
                  file_name = 'precip_sums_try',
                  format = 'CSV',
                  output_folder = 'climate_stats',
                  name = 'precip_sum_mean')

{'state': 'READY', 'description': 'precip_sums_try', 'creation_timestamp_ms': 1653069937204, 'update_timestamp_ms': 1653069937204, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_FEATURES', 'id': '2JSBXBHBHK4V24J5NFP7U3HZ', 'name': 'projects/earthengine-legacy/operations/2JSBXBHBHK4V24J5NFP7U3HZ'}


In [ ]:
## ----------------------------- ##
## 4. IMPORT MONTHLY PRECIP DATA ##
## ----------------------------- ##

years = np.arange(1987,2021)
months = np.arange(3,11)
sum_month_pr = []

for i in years:
  for m in months:
    if (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m==10):
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-31')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels
    else: 
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-30')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels

sum_month_precip = ee.ImageCollection(sum_month_pr) #convert list of image to image collection for zonal stats command


In [9]:
## IMPORT MONTHLY ET DATA ##

et_irrig = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_irrig.append(et)

et_irrig = ee.ImageCollection(et_irrig)

et_cut = []

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-04-01'), str(y)+'-9-30')
  et_march = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-03-31')
  et_march = et_march.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-03')})
  et_oct = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-10-01'), str(y)+'-10-31')
  et_oct = et_oct.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-10')})
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_cut.append(et)
  et_cut.append(et_march)
  et_cut.append(et_oct)

et_cut = ee.ImageCollection(et_cut)
  

In [11]:

## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/precip_stats.csv')
emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats.csv')
emap.zonal_statistics(irrig_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/month_precip_stats.csv')
emap.zonal_statistics(sum_month_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/et_cut.csv')
emap.zonal_statistics(et_cut, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/maxtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/irrig_precip_stats.csv
Computing statistics ...


Exception: ignored

In [22]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')
irrig_pre = pd.read_csv('climate_stats/irrig_precip_stats.csv')
et_irrig = pd.read_csv('climate_stats/et.csv')
et_cut_ir = pd.read_csv('climate_stats/et_cut.csv')

names = et_cut_ir['WaterRight']

cut = pd.DataFrame(et_cut_ir['WaterRight'])
for i in years:
  fill = str(i)
  cut[fill] = et_cut_ir[et_cut_ir.columns[et_cut_ir.columns == (str(i)+'_et')]]

display(cut)

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['Precip_mm'] = precip.iloc[i,1:35].values
  df['Max_temp'] = max_temp.iloc[i,0:34].values
  df['et_whole'] = et_irrig.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


,WaterRight,1987,1988,1989,1990,1991,1992,1993,1994,1995,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Andrews,0.738347,0.722437,0.586516,0.774692,0.771539,0.840096,0.686474,0.827853,0.756042,...,0.725641,0.798259,0.822650,0.841210,0.746635,0.817068,0.782396,0.827454,0.773377,0.771053
1,Atwell,0.911433,0.757452,1.038284,0.889469,0.965961,0.777396,0.880189,0.968748,0.961381,...,0.844044,0.958025,0.681203,0.852056,0.956136,0.980913,1.001046,0.863300,0.876567,0.788324
2,Ballentyne,0.852817,0.890583,0.895612,0.930805,0.873681,0.896400,0.746552,0.874973,0.880809,...,0.690901,0.750701,0.728641,0.765250,0.764683,0.818744,0.748486,0.798498,0.722156,0.753462
3,Barber,0.887926,0.818039,0.854366,0.888915,0.942506,0.907564,0.782186,1.069858,1.000258,...,0.858523,0.929648,0.974985,1.075113,1.023822,1.129468,1.017852,0.534945,0.686583,0.802394
4,Bates,0.767621,0.754882,0.581457,0.685497,0.790001,0.690025,0.654684,0.740638,0.775197,...,0.718174,0.830277,0.793234,0.824497,0.790054,0.819313,0.840282,0.781373,0.741666,0.684296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Thomas Aiken,1.165441,1.172600,1.038744,1.032624,1.089479,1.129904,0.868714,1.103753,1.022119,...,0.752728,0.815053,0.867930,0.881664,0.908503,0.944922,0.851436,0.917933,0.878956,0.869190
60,Thurman Mill,0.849821,0.889409,0.914166,0.883931,0.833745,0.854526,0.701805,0.811323,0.838167,...,0.759724,0.799190,0.795028,0.836755,0.842143,0.870701,0.828585,0.837485,0.822237,0.822383
61,Upper Centerpoint,0.673644,0.766561,0.620031,0.771872,0.752010,0.776875,0.767011,0.858466,0.779173,...,0.659978,0.791347,0.711161,0.759960,0.725534,0.692221,0.764767,0.781090,0.693890,0.704431
62,Wagner,0.696458,0.658753,0.675133,0.685217,0.549909,0.589289,0.567715,0.806028,0.663881,...,0.671933,0.818889,0.751563,0.825714,0.764789,0.767051,0.833249,0.808667,0.821409,0.721642


In [15]:
## --------------- ## 
## COMPARE ET DATA ##
## --------------- ## 

years = np.arange(1987,2021)
et_whole = pd.read_csv('climate_stats/et.csv').drop(columns = ['Shape_Area', 'system:index', 'Shape_Leng'], axis=1)
et_cut = pd.read_csv('climate_stats/et_cut.csv')


names = et_whole['WaterRight']
et = []

march = pd.DataFrame(et_cut['WaterRight'])
october = pd.DataFrame(et_cut['WaterRight'])
cut = pd.DataFrame(et_cut['WaterRight'])

for i in years:
  fill = str(i)
  march[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-03_et')]]
  october[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-10_et')]]
  cut[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'_et')]]

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['et_whole'] = et_whole.iloc[i,0:34].values
  df['et_cut'] = cut.iloc[i,1:35].values
  df['march'] = march.iloc[i,1:35].values
  df['october'] = october.iloc[i,1:35].values
  et.append(df)

et = pd.concat(et)

et['diff'] = et['et_whole'] - et['et_cut']

avg = et.groupby('DIV_NAME')['et_whole', 'et_cut', 'diff', 'march', 'october'].mean()
avg.to_csv('climate_stats/et_avgs.csv')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [ ]:
## ------------------------- ##
## COPY ET DATA to Cloud API ##
## ------------------------- ##

# folder from where to copy
src_folder = "projects/earthengine-legacy/assets/users/dgketchum/ssebop/boise"
# folder where to copy
dest_folder = "projects/ee-bridgetbittmann/assets/ssebop"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)

In [ ]:
## ------------------------------------------- ##
## COPY ET DATA from Cloud API to Earth Engine ##
## ------------------------------------------- ##

# folder from where to copy
src_folder = "projects/ee-bridgetbittmann/assets/ssebop"
# folder where to copy
dest_folder = "projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)